In [19]:
from arsenic import browsers

In [23]:
browsers.Chrome

Init signature: browsers.Chrome(**overrides)
Docstring:      <no docstring>
Source:        
class Chrome(Browser):
    defaults = {"browserName": "chrome"}
File:           ~/.virtualenvs/web-scraping-with-asyncio-python-BkoSTc6q/lib/python3.9/site-packages/arsenic/browsers.py
Type:           type
Subclasses:     


In [ ]:
!pip install structlog

In [44]:
%%writefile async_scrape.py

import os
import asyncio
from arsenic import get_session, keys, browsers, services
import pandas as pd
from requests_html import HTML
import itertools
import re
import time
import pathlib

import logging
import structlog # pip install structlog

def set_arsenic_log_level(level = logging.WARNING):
    # Create logger
    logger = logging.getLogger('arsenic')
    
    # We need factory, to return application-wide logger
    def logger_factory():
        return logger
    
    structlog.configure(logger_factory=logger_factory)
    logger.setLevel(level)



# /en/fabric/71433-genevieve-floral-by-crystal_walen
async def extract_id_slug(url_path):
    regex = r"^[^\s]+/(?P<id>\d+)-(?P<slug>[\w_-]+$"
    group = re.match(regex, url_path)
    if not group:
        return None, None
    return group['id'], group['slug']
    
    
async def get_links(body_content):
    html_r = HTML(html=body_content)
    fabric_links = [x for x in list(html_r.links) if x.startswith("/en/fabric")]
    datas = []
    for path in fabric_links:
        id_, slug_ = await extract_id_slug(path)
        data = {
            'id': id_,
            'slub': slug_,
            'path': path,
            'scraped': 0 # True / False -> 1 / 0
        }
        datas.append(data)
        
    return datas

async def scraper(url):
    service = services.Chromedriver()
    browser = browsers.Chrome(chromeOptions={
        'args': ['--headless', '--disable-gpu']
    })
    
    async with get_session(service, browser) as session:
        await session.get(url)
        body = await session.get_page_source()
        # print(body)
        return body
    
async def store_links_as_df_pickle(datas=[], name='links.pkl'):
    df = pd.DataFrame(datas)
    df.set_index('id', drop=True, inplace=True)
    df.to_pickle(name)
    return df

    
async def run(url):
    body_content = await scraper(url)
    links = await get_links(body_content)
    df = await store_links_as_df_pickle(links)
    return df
    
    
if __name__ == "__main__":
    url = 'https://www.spoonflower.com/en/shop?on=fabric'
    set_arsenic_log_level()
    # loop = asyncio.get_event_loop()
    # results = loop.run_until_complete(scraper(url))
    
    df = asyncio.run(run(url))
    print(df.head())

Overwriting async_scrape.py


In [45]:
!python3 async_scrape.py

Starting ChromeDriver 98.0.4758.102 (273bf7ac8c909cde36982d27f66f3c70846a3718-refs/branch-heads/4758@{#1151}) on port 57093
Only local connections are allowed.
Please see https://chromedriver.chromium.org/security-considerations for suggestions on keeping ChromeDriver safe.
ChromeDriver was started successfully.
2022-06-25 14:06.32 [error    ] error                          data={'error': 'invalid argument', 'message': 'invalid argument: unrecognized capability: chromeOptions', 'stacktrace': '#0 0x55863079dd33 <unknown>\n#1 0x5586304ff518 <unknown>\n#2 0x558630515138 <unknown>\n#3 0x558630556b8b <unknown>\n#4 0x5586305565c6 <unknown>\n#5 0x558630557c2d <unknown>\n#6 0x558630552083 <unknown>\n#7 0x558630527c0c <unknown>\n#8 0x558630528cb5 <unknown>\n#9 0x5586307cb106 <unknown>\n#10 0x5586307e292d <unknown>\n#11 0x5586307cd06e <unknown>\n#12 0x5586307e32b4 <unknown>\n#13 0x5586307c0e1b <unknown>\n#14 0x5586307fc908 <unknown>\n#15 0x5586307fca88 <unknown>\n#16 0x5586308164c6 <unknown>\n#1